#  Calibration

The objective is to calibrate a camera using a projection model
center (pinhole model). The calibration will be done with the linear model allowing the representation of the camera 

This calibration technique will make it possible to calculate the matrix M from a
set of points given in (mypoints.csv)
The data file contains the 3D/2D correspondence list, the 3D point of the world
real and its 2D projection on the image plane. The first three values are the
coordinates (X, Y, Z) of the 3D point in the image and the next two are the
coordinates (u, v) of the 2D point in the image

## Procedure


Each point gives us 2 plane equations and we have 11 unknowns (5 intrinsic and
6 extrinsic) so a minimum of 6 points are then required.
We construct a system of linear equations that we can solve using the
least square techniques.
We estimate the mij (intrinsic and extrinsic parameters)

In [43]:
import numpy as np
import pandas as pd

In [44]:
def myCalibration(x):
    np.set_printoptions(suppress=True)
    A = []
    B = []
    for d in x:
        A.append([d[0], d[1], d[2], 1, 0, 0, 0, 0, d[3]*d[0]*(-1), d[3]*d[1]*(-1), d[3]*d[2]*(-1)])
        A.append([0, 0, 0, 0, d[0], d[1], d[2], 1,d[4]*d[0]*(-1), d[4]*d[1]*(-1), d[4]*d[2]*(-1)])
        B.append(d[3])
        B.append(d[4])
    B = np.array(B)
    A = np.array(A)
    print("\n************************************************************\n")
    print(A.shape," \nA = ",A)
    print("\n************************************************************\n")
    print(B.shape," \nB = ",B)
    
    
    s=np.matmul(np.transpose(A),A)
    s=np.linalg.pinv(s)
    s=np.matmul(s,np.transpose(A))
    S=np.matmul(s,np.transpose(B))
    print("\n************************************************************\n")
    print(S.shape," \nS = ",S)
    
    
    M1 = np.array([S[0], S[1], S[2]])
    M2 = np.array([S[4], S[5], S[6]])
    M3 = np.array([S[8], S[9], S[10]])
    M14 = S[3]
    M24 = S[7]
    M34 = 1
    M3t = np.transpose(M3)
    
    u0 = M1@M3t
    v0 = M2@M3t
    print("\n************************************************************\n")
    print("u0 = ", u0, "\nv0 = ", v0)
    
    
    au = np.linalg.norm(np.cross(M1, M3))
    av = np.linalg.norm(np.cross(M2, M3))
    print("\n************************************************************\n")
    print("au = ", au,"\nav = ", av)
    
    
    tx = (M14-u0*M34)/au
    ty = (M24-v0*M34)/av
    tz = 1
    r1 = (M1-u0*M3)/au
    r2 = (M2-v0*M3)/av
    r3 = M3
    
    
    MI = np.array([[au, 0, u0, 0],
                   [0, av, v0, 0],
                   [0, 0, 1, 0]])
    ME = np.array([[r1[0], r1[1], r1[2], tx],
                   [r2[0], r2[1], r2[2], ty],
                   [r3[0], r3[1], r3[2], tz]])
    print("\n************************************************************\n")
    print("MI\n", MI) 
    print("\n************************************************************\n")
    print("ME\n", ME)
    M = np.array([[S[0],S[1], S[2], S[3]],
          [S[4],S[5], S[6], S[7]],
          [S[8],S[9], S[10], M34]])
    print("\n************************************************************\n")
    print("M = ",M)

In [45]:
np.set_printoptions(suppress=True)
mydata = pd.read_csv("mypoints.csv")
data=np.array(mydata)
cam=data[0:13]
print(cam.shape,"\n",cam)


(13, 7) 
 [[140 20 0 655.0 759.5 391.5 '735.5;;;']
 [120 20 0 839.5 792.5 575.0 '761;;;']
 [140 0 0 639.5 948.0 374.5 '919.5;;;']
 [120 0 0 828.0 972.0 563.0 '938;;;']
 [60 -20 0 1306.5 1207.5 1055.5 '1158.5;;;']
 [40 -20 0 1437.5 1221.5 1195.0 '1170;;;']
 [20 -20 0 1553.9 1234.5 1320.5 '1181.5;;;']
 [60 -40 0 1308.5 1373.5 1057.0 '1324;;;']
 [40 -40 0 1439.5 1378.5 1197.5 '1327.5;;;']
 [20 -40 0 1557.0 1383.0 1324.0 '1331;;;']
 [60 -60 0 1310.5 1541.5 1061.5 '1491.5;;;']
 [40 -60 0 1442.5 1536.5 1201.5 '1486.5;;;']
 [20 -60 0 1559.0 1532.5 1327.5 '1481;;;']]


In [47]:
myCalibration(cam)


************************************************************

(26, 11)  
A =  [[    140.      20.       0.       1.       0.       0.       0.       0.
   -91700.  -13100.      -0.]
 [      0.       0.       0.       0.     140.      20.       0.       1.
  -106330.  -15190.      -0.]
 [    120.      20.       0.       1.       0.       0.       0.       0.
  -100740.  -16790.      -0.]
 [      0.       0.       0.       0.     120.      20.       0.       1.
   -95100.  -15850.      -0.]
 [    140.       0.       0.       1.       0.       0.       0.       0.
   -89530.      -0.      -0.]
 [      0.       0.       0.       0.     140.       0.       0.       1.
  -132720.      -0.      -0.]
 [    120.       0.       0.       1.       0.       0.       0.       0.
   -99360.      -0.      -0.]
 [      0.       0.       0.       0.     120.       0.       0.       1.
  -116640.      -0.      -0.]
 [     60.     -20.       0.       1.       0.       0.       0.       0.
   -78390.   26